# MODEL EVALUATION NOTEBOOK

In this notebook we'll use several tools to visualize the inner working of our GBM models, how & why they make their predictions.
For this we'll use feature importances visualization and the package shap that allow us to access many very useful viz.

### What are shapley values ?

Shapley values were conceived to try and answer a theoretical game theory question "how can we find each player's marginal contribution, averaged over every possible sequence in which the players could have been added to the group ?" /
In our context of evaluating our model features importances, we can see that shapley values can be a very valuable asset to understand each feature marginal effect on our model predictions.

The equation they find to evaluate this effect satisfies three axioms of credit-attribution : 
* If a player never adds any marginal value, their payoff portion should be 0 (Dummy player)
* If two players always add the same marginal value to any subset to which they're added, their payoff portion should be the same (Substitutability)
* If a game is composed of two subgames, you should be able to add the payoffs calculated on the subgames and it should match the payoff of the full game (Additivity)

## IMPORTS

In [ ]:
import os, joblib
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import shap
import matplotlib.pylab as pl
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

shap.initjs()

## UTILS

In [ ]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder 

def cat_encoding(dataframe):
    cat = dataframe.columns[1:20]
    for feature in cat:
        le = LabelEncoder()
        le.fit(dataframe[feature])
        dataframe[feature] = le.transform(dataframe[feature])
    return dataframe

def feature_engineering(dataframe):
    dataframe = cat_encoding(dataframe)
    features = dataframe.columns[1:31]
    return dataframe, features

## CONFIG

In [ ]:
config = {
    "MODEL_PATH" : "D:/Documents/GitHub/gbm_pipeline/task/TPS-MAR2021/model_saved/XGB_CL_model_1.joblib.dat",
    "TRAIN_PATH" : "../input/tabular-playground-series-mar-2021/train.csv",
    "TEST_PATH" : "../input/tabular-playground-series-mar-2021/test.csv",
    "TARGET_VAR" : "target"
}

## LOADING DATA

In [ ]:
df_train = pd.read_csv(config["TRAIN_PATH"])
df_test = pd.read_csv(config["TEST_PATH"])

In [ ]:
df_train, features_train = feature_engineering(df_train)
df_test, features_test = feature_engineering(df_test)

In [ ]:
target_train = df_train[config["TARGET_VAR"]].values
df_train = df_train[features_train]
train_x, valid_x, train_y, valid_y = train_test_split(df_train, target_train, test_size=0.2, random_state=95)

In [ ]:
train = xgb.DMatrix(train_x, label=train_y)
valid = xgb.DMatrix(valid_x, label=valid_y)
test = xgb.DMatrix(df_test[features_test])

## TRAINING A MODEL

In [ ]:
params = {
    "objective": "binary:logistic",
    "eval_metric" : "auc",
    "seed": 95,
    'tree_method': "gpu_hist",
    'predictor': 'gpu_predictor',
    "use_label_encoder" : False,
    "n_estimators" : 100000,
    'max_bin' : 64,
    "max_depth": 12, #Max should correspond to max number of features (probably ?),
    'alpha' : 11.607239831188968,
    'gamma' : 2.1593805822598444,
    "learning_rate": 0.02,
    "colsample_bytree": 0.8016656211574054,
    "subsample": 0.983461992112787,
    "reg_alpha" : 1.7306711078859136,
    "min_child_weight": 9.417969426623086,
    "n_jobs": 2
}

In [ ]:
#model = xgb.train(params, train, 400)
#valid_oof = model.predict(valid)
#temp_test = model.predict(test)
#auc = roc_auc_score(valid_y, valid_oof)
#print('AUC score %.6f' % auc)

## FITTING A MODEL

In [ ]:
model = xgb.XGBClassifier(**params)
model.fit(
    train_x, 
    train_y, 
    eval_set=[(valid_x, valid_y)], 
    early_stopping_rounds=200, 
    verbose = 1000
)
valid_oof = model.predict(valid_x)
temp_test = model.predict(df_test[features_test])
auc = roc_auc_score(valid_y, valid_oof)

In [ ]:
print('AUC score %.6f' % auc)

### XGB boost plot tools

In [ ]:
xgb.plot_importance(model)

## SHAP

In [ ]:
%%time

explainer = shap.TreeExplainer(model)
shap_values = explainer(valid_x)

In [ ]:
shap_values.shape

In [ ]:
%%time

shap.plots.beeswarm(shap_values, max_display=35)

In [ ]:
shap.plots.bar(shap_values, max_display=35)

## VIZ INDIVIDUAL DATA PREDS

In [ ]:
shap.plots.waterfall(shap_values[2])

In [ ]:
shap.plots.force(shap_values[0])